In [2]:
import pandas as pd
import json

In [4]:
with open("datasets/snli/snli_1.0/snli_1.0_test_batch.jsonl", "r") as f:
    # Read the JSON lines into a list
    json_list = [json.loads(line) for line in f]

df = pd.DataFrame(json_list)
df.head()

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,"[neutral, contradiction, contradiction, neutra...",2677109430.jpg#1,neutral,2677109430.jpg#1r1n,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church has cracks in the ceiling.,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...
1,"[entailment, entailment, entailment, neutral, ...",2677109430.jpg#1,entailment,2677109430.jpg#1r1e,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,The church is filled with song.,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...
2,"[contradiction, contradiction, contradiction, ...",2677109430.jpg#1,contradiction,2677109430.jpg#1r1c,This church choir sings to the masses as they ...,( ( This ( church choir ) ) ( ( ( sings ( to (...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,A choir singing at a baseball game.,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...
3,"[neutral, neutral, neutral, neutral, neutral]",6160193920.jpg#4,neutral,6160193920.jpg#4r1n,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is young.,( ( The woman ) ( ( is young ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...
4,"[entailment, entailment, contradiction, entail...",6160193920.jpg#4,entailment,6160193920.jpg#4r1e,"A woman with a green headscarf, blue shirt and...",( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,The woman is very happy.,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...


In [5]:
df = df[["sentence1", "sentence2", "gold_label"]]
df.head()

,sentence1,sentence2,gold_label
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment


In [6]:
import random
def random_label():
    return random.choice(["entailment", "contradiction", "neutral"])

In [ ]:
# create a copy dataframe
random_results = df.copy()

random_results["result"] = random_results.apply(lambda x: random_label(), axis=1)

,sentence1,sentence2,gold_label,result
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral,entailment
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral,contradiction
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment,entailment


In [9]:
random_results.head()   

,sentence1,sentence2,gold_label,result
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral,entailment
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral,contradiction
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment,entailment


In [10]:
accuracy = (random_results["result"] == random_results["gold_label"]).mean()
accuracy

np.float64(0.324)

In [11]:
random_results.to_csv("random_results.csv")

In [12]:
from transformers import pipeline

In [17]:
classifier = pipeline("text-classification", model="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli", tokenizer="MoritzLaurer/DeBERTa-v3-base-mnli-fever-anli")

# Create a results dataframe
new_model_results = df.copy()

# Function to process a batch of examples
def process_batch(batch_df):
    inputs = []
    for _, row in batch_df.iterrows():
        # Format sentences in the required format for the model
        input_text = f"{row['sentence1']}</s>{row['sentence2']}"
        inputs.append(input_text)
    
    # Run the model on the batch
    results = classifier(inputs)
    
    # Extract labels
    labels = [item['label'].lower() for item in results]
    return labels

# Process in batches to avoid memory issues
batch_size = 16
results = []

for i in range(0, len(df), batch_size):
    batch_df = df.iloc[i:i+batch_size]
    batch_results = process_batch(batch_df)
    results.extend(batch_results)

# Add results to dataframe
new_model_results["result"] = results

new_model_results.head()

Device set to use mps:0


,sentence1,sentence2,gold_label,result
0,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral,neutral
1,This church choir sings to the masses as they ...,The church is filled with song.,entailment,entailment
2,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction,contradiction
3,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral,neutral
4,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment,entailment


In [18]:
# Calculate accuracy
accuracy = (new_model_results["result"] == new_model_results["gold_label"]).mean()
print(f"Model accuracy: {accuracy:.4f}")

# Save results
new_model_results.to_csv("roberta_results.csv")

Model accuracy: 0.8520


In [21]:
test_df = pd.read_csv("csv/random_nli_model_answers.csv")
accuracy = (test_df["result"] == test_df["gold_label"]).mean()
print(f"Model accuracy: {accuracy:.4f}")

Model accuracy: 0.3273
